In [1]:
import pandas as pd

fileName = 'spareParts.xlsx'

# Read the Excel file into a DataFrame
df = pd.read_excel(fileName)

# Get column names
# column_names = df.columns

# Define columns to keep
columns_to_keep = ['TrainNumber', 'System', 'Date', 'Quantity']  

# Keep specific columns
df = df.loc[:, columns_to_keep]

# Assuming 'Date' column is already in datetime format
df['Date'] = pd.to_datetime(df['Date'])

# Set the index to the 'Date' column
df.set_index('Date', inplace=True)

# Group by 'System' and resample by '1M' (monthly frequency), summing the 'Quantity' column
feature_monthly_quantity_df = df.groupby('System').resample('1M')['Quantity'].sum().reset_index()

# Reset index to make the DataFrame more manageable
feature_monthly_quantity_df.reset_index(drop=True, inplace=True)

# feature_monthly_quantity_df.to_csv('sparePartsDataframe.csv', index=False)

print(feature_monthly_quantity_df)


ModuleNotFoundError: No module named 'pandas'

In [24]:
from darts import TimeSeries
from sklearn.preprocessing import LabelEncoder
from darts.models import XGBModel,CatBoostModel

label_encoder=LabelEncoder()

feature_monthly_quantity_df['SystemEncoded'] = label_encoder.fit_transform(feature_monthly_quantity_df['System'])

In [25]:
def train_test_split_last_n_rows(df, percentage):
    # Define a function to split the last n rows for each product
    def split_last_n_rows_train(group):
        n_rows = int(len(group) * (1 - percentage))
        return group.iloc[:n_rows]

    def split_last_n_rows_test(group):
        n_rows = int(len(group) * percentage)
        return group.iloc[-n_rows:]

    # Apply the split function to each product group
    train_df = df.groupby('System', group_keys=False).apply(
        split_last_n_rows_train)

    test_df = df.groupby('System', group_keys=False).apply(
        split_last_n_rows_test)

    train_df = train_df.groupby('System', group_keys=False).apply(
        lambda group: group.reset_index(drop=True))
    return train_df, test_df

In [26]:
train, test = train_test_split_last_n_rows(feature_monthly_quantity_df, 0.2)

C:\Users\Waleed Bin Osama\AppData\Local\Temp\ipykernel_23492\537614829.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  train_df = df.groupby('System', group_keys=False).apply(
C:\Users\Waleed Bin Osama\AppData\Local\Temp\ipykernel_23492\537614829.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  test_df = df.groupby('System', group_keys=False).apply(
C:\Users\Waleed Bin Osama\AppData\Local\Temp\ipyk

In [27]:
future_covariates_df=pd.concat([train,test],ignore_index=True)

# future_features=['base_price','is_holiday','day_of_week','week_of_month','month_of_year','days_till_black_friday','days_till_christmas','days_till_summer','days_till_winter','is_promotion','days_till_thanksgiving','days_till_independence_day','base_price_rolling_3','base_price_rolling_7','base_price_rolling_30']
# past_features=['base_price','is_holiday','day_of_week','week_of_month','month_of_year','days_till_black_friday','days_till_christmas','days_till_summer','days_till_winter','is_promotion', 'days_till_thanksgiving','days_till_independence_day','base_price_rolling_3','base_price_rolling_7','base_price_rolling_30']



train_time_series = TimeSeries.from_group_dataframe(train, group_cols = "SystemEncoded", time_col = 'Date', fill_missing_dates = True, freq='M', value_cols=['Quantity'])
# future_covariates_series2=TimeSeries.from_group_dataframe(train,group_cols="product_item_sku_id_encoded",time_col='creation_date',fill_missing_dates=False,freq=fr,value_cols=future_features)
# past_covariates_series2=TimeSeries.from_group_dataframe(train,group_cols="product_item_sku_id_encoded",time_col='creation_date',fill_missing_dates=False,freq=fr,value_cols=past_features)
#Model Pipeline


model_params = {
    'lags': [-1, -3],
    # 'lags_future_covariates': [0],
    # 'lags_past_covariates' : [-1,-3,-5,-10],
    'use_static_covariates': True,
    'output_chunk_length': 1,    
}


model=XGBModel(**model_params)

c:\Users\Waleed Bin Osama\OneDrive - FAST National University\Waleed Bin Osama\Work\Flutter\third_app\third_app\.fypvenv\Lib\site-packages\darts\timeseries.py:4674: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  resampled_time_index = resampled_time_index.asfreq(freq)
c:\Users\Waleed Bin Osama\OneDrive - FAST National University\Waleed Bin Osama\Work\Flutter\third_app\third_app\.fypvenv\Lib\site-packages\darts\timeseries.py:4674: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  resampled_time_index = resampled_time_index.asfreq(freq)
c:\Users\Waleed Bin Osama\OneDrive - FAST National University\Waleed Bin Osama\Work\Flutter\third_app\third_app\.fypvenv\Lib\site-packages\darts\timeseries.py:4674: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  resampled_time_index = resampled_time_index.asfreq(freq)
c:\Users\Waleed Bin Osama\OneDr

In [28]:
df1 = pd.DataFrame(columns = ['System', 'Quantity'])
df2 = pd.DataFrame(columns = ['System'])
df2['System'] = feature_monthly_quantity_df['System'].unique()
for x in range(max(feature_monthly_quantity_df['SystemEncoded'])):
    model.fit(series = train_time_series[x])
    pred = model.predict(3, series = train_time_series[x])
    quantity = pred.values()[2][0]
    quantity = max(round(quantity), 0)
    df1.loc[x] = [df2['System'][x], quantity]
    
    # df1 = df1.append({'System': df2['System'][x], 'Quantity': quantity})
print(df1)

df1.to_csv('sparePartsPrediction.csv', index=False)

               System  Quantity
0      Air_Compressor         0
1        Bogie_System         3
2             Coupler         0
3   Current_Collector         9
4                 DCU         5
5                Door         2
6          Driver_Cab         0
7                 FAS         0
8            HVAC_Cab         0
9         HVAC_Saloon         3
10           Interior         0
11  Internal_Lighting         3
12          Lightning         2
13       Node_Circuit         0
14                PIS        13
15   Pneumatic_system         1
16                SIV         1
17               TCMS         0
18        Under_frame        29


In [12]:
# df1 = pred.pd_dataframe()
# df1['Quantity'] = df1['Quantity'].apply(lambda x: max(0, round(x))).astype(int).tolist()
# print(df1)

component   Quantity
Date                
2023-11-30        29
